<a href="https://colab.research.google.com/github/karlssoj/compvis/blob/Exempel4_cats_dogs_classification_transfer_learning/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> Att träna in en egen klassificeringsmodell, som klassificerar hund och katt, med EfficientNetB0 och överföringsinlärning </h1>

**Laddar ner cats & dogs dataset**

In [ ]:
!wget  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip

**Packar upp ZIP-filen**

In [ ]:
!unzip kagglecatsanddogs_5340.zip

**Tar bort de "korrupta" bildfilerna**

In [3]:
!rm /content/PetImages/Cat/666.jpg
!rm /content/PetImages/Dog/11702.jpg

**Delar upp datasetet i test-, validerings- och träningsbilder**

TESTBILDERNA används under själva inträningen för att träna modellen

VALIDERINGSBILDERNA används vid slutet av varje epoch (efter varje gång träningen gått igenom alla bilder en gång) för att utvärdera prestandan (genom att beräkna loss och accuracy)

TESTBILDERNA används efter avslutad inträning för att evaluera hur väl bildklassificeringsmodellen presterar på okänt data (testbilderna används inte under träningen)





In [ ]:
!pip install split-folders

In [7]:
import splitfolders

# Path to original dataset
input_folder = "/content/PetImages"  # The folder containing "Cat" and "Dog" subfolders

# Split into train (80%), validation (10%), and test (10%)
splitfolders.ratio(input_folder, output="PetImages_Split", seed=42, ratio=(0.8, 0.1, 0.1), group_prefix=None)

Copying files: 25000 files [00:08, 3062.12 files/s]


**Skapar ett ImageDataGenerator-objekt för att hantera träningsdatasetet och som bl.a. definierar hur bilderna ska pre-processeras och normaliseras. I detta fall använder vi EfficientNet som kräver en viss typ av pre-processering och normalisering (därav preprocessing_function = preprocess_input)**

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
image_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

**Definierar bl.a. varifrån tränings-, validerings- och testbilderna ska laddas in**

In [ ]:
train_image_gen = image_gen.flow_from_directory(
    "/content/PetImages_Split/train",
    target_size=(224, 224),#EfficienNet kräver juse denna upplösning
    batch_size=32,#Hur många bilder som ska matas per gång
    class_mode="binary",
    shuffle=True
)

validation_image_gen = image_gen.flow_from_directory(
    "/content/PetImages_Split/val",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary"
)

test_image_gen = image_gen.flow_from_directory(
    "/content/PetImages_Split/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)

**Sätter upp CNN-nätet för träning. Vi använder oss av den förintränade EfficientNetB0-modellen. Vi "fryser" alla faltningslager, dvs. vi vill inte träna in på nytt olika lösryckte egenskaper utan vi återanvänder samma egenskaper som tränats på ImageNet datasetet. Det ända vi vill träna är klassificeringslagret, dvs. hur de olika lösryckte egenskaperna ska kombineras för att det ska känneteckna en hund eller katt**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3)
)

# Frys basmodellen så att dess vikter inte förändras under träningen
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation="relu")(x)
x = Dropout(0.5)(x)  # För att minska överanpassning

#Output är i detta fall enbart en neuron, dvs. vad är sannolikheten för att bilden
#innehåller en hund. 1=hund, 0=katt
output_layer = Dense(1, activation="sigmoid")(x)  # Binary classification
model = Model(inputs=base_model.input, outputs=output_layer)

<b> "Kompilerar" modellen. Optimizer är den metod man vill använda för att optimera alla vikter i nätet. Adam är en variant av gradient descent. Loss definierar "loss function", dvs. vad man vill använda för metod för att beräkna feluppskattningen under inträningen.

In [13]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",  # Correct loss function for 2-class problem
    metrics=["accuracy"]
)

In [ ]:
model.summary()

<b> Startar själva inträningen på testdataset:et. Epochs definierar hur många ggr inlärningsalgoritmen kommer att gå igenom hela dataset:et.Här definierar vi även pathen till valideringsdatasetet så att modellen ska kunna utvärderas på okända bilder efter varje epoch

In [ ]:
# Train the model with validation
results = model.fit(
    train_image_gen,
    validation_data=validation_image_gen,
    epochs=10
)

In [ ]:
plt.plot(results.history['accuracy'])

<b> Vi evaluerar hur väl den intränade klassificeringsmodellen presterar mot test-datasetet, dvs. vi testar den intränade modellen på data den inte sett tidigare

In [ ]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(test_image_gen)
print(f"Test Accuracy: {test_acc:.4f}")